In [1]:
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/semantic_search/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineG

# Finetune alea-institute/kl3m-doc-micro-uncased-001 with chunked_opinion:relevant:irrelevant query triples

1. The opinions were pre-chunked to at most 512 tokens
2. The queries were generated using gpt-4o
3. The finetuning is run in Google Colab with a T4 GPU

# Import Libraries

In [3]:
import pandas as pd

%pip install datasets -q
from datasets import Dataset

from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
    SentenceTransformerModelCardData,
)
from sentence_transformers.losses import MultipleNegativesRankingLoss
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.evaluation import TripletEvaluator



# Load the data

In [4]:
train = pd.read_csv(f"{path}outputs/5.finetune_train_512.csv")
val = pd.read_csv(f"{path}outputs/5.finetune_val_512.csv")

# Take a look at the token count distribution

In [5]:
train["chunk_size"].describe()

,chunk_size
count,2828.000000
mean,432.561174
std,75.628180
min,30.000000
25%,435.000000
50%,458.000000
75%,471.000000
max,480.000000


In [6]:
val["chunk_size"].describe()

,chunk_size
count,489.000000
mean,416.613497
std,91.049189
min,22.000000
25%,418.000000
50%,451.000000
75%,468.000000
max,480.000000


# Generate dataset objects

In [7]:
cols = ["chunked_opinion", "relevant", "irrelevant"]
train = train[cols]
val = val[cols]

In [8]:
renames = {"chunked_opinion":"anchor", "relevant":"positive", "irrelevant":"negative"}
train = train.rename(columns=renames)
val = val.rename(columns=renames)

In [9]:
train_dataset = Dataset.from_pandas(train, split="train")
eval_dataset = Dataset.from_pandas(val, split="test")

In [10]:
train_dataset

Dataset({
    features: ['anchor', 'positive', 'negative'],
    num_rows: 2828
})

In [11]:
eval_dataset

Dataset({
    features: ['anchor', 'positive', 'negative'],
    num_rows: 489
})

# Train the model

In [12]:
BASE_MODEL = "alea-institute/kl3m-doc-micro-uncased-001"
BASE_MODEL_NAME = BASE_MODEL.split("/")[-1]

In [13]:
# Load the model
model = SentenceTransformer(
    BASE_MODEL,
    model_card_data=SentenceTransformerModelCardData(
        language="en",
        license="apache-2.0",
        model_name=f"{BASE_MODEL_NAME} trained on triplets",
    )
)

# Define loss function
loss = MultipleNegativesRankingLoss(model)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [14]:
# Specify training arguments
args = SentenceTransformerTrainingArguments(
    output_dir=f"{path}models/{BASE_MODEL_NAME}",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    warmup_ratio=0.1,
    fp16=True,
    bf16=False,
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # MultipleNegativesRankingLoss benefits from no duplicate samples in a batch
    # Optional tracking/debugging parameters:
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    report_to="none"

)

In [15]:
# Evaluate the base model before finetuning
dev_evaluator = TripletEvaluator(
    anchors=eval_dataset["anchor"],
    positives=eval_dataset["positive"],
    negatives=eval_dataset["negative"],
    name="dev",
)
dev_evaluator(model)

{'dev_cosine_accuracy': 0.9018405079841614}

In [16]:
# Create a trainer & train
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=loss,
    evaluator=dev_evaluator,

)
trainer.train()

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss,Validation Loss,Dev Cosine Accuracy
100,No log,0.509717,0.985685
200,No log,0.406718,0.991820
300,No log,0.391953,0.991820


TrainOutput(global_step=354, training_loss=0.4911390401549258, metrics={'train_runtime': 398.4099, 'train_samples_per_second': 7.098, 'train_steps_per_second': 0.889, 'total_flos': 0.0, 'train_loss': 0.4911390401549258, 'epoch': 1.0})

In [17]:
# Evaluate the finetuned model
dev_evaluator = TripletEvaluator(
    anchors=eval_dataset["anchor"],
    positives=eval_dataset["positive"],
    negatives=eval_dataset["negative"],
    name="dev",
)
dev_evaluator(model)

{'dev_cosine_accuracy': 0.9938650131225586}

In [18]:
# Save the trained model
model.save_pretrained(f"{path}models/{BASE_MODEL_NAME}/final")

In [19]:
# Push the trained model to hub
model.push_to_hub(f"{BASE_MODEL_NAME}_triples", private=True)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/12.0M [00:00<?, ?B/s]

'https://huggingface.co/rachelFLP/kl3m-doc-micro-uncased-001_triples/commit/606e7765259cdb440bd677fea07c330f462e593d'